# Open Access - Google Scholar
La base de datos oaudea.json que fue creada con: [oaudea.ipynb](./oaudea.ipynb)

Está basada en la combinación de:

* Web of Science con Scopus y Scielo (Web of science) para la extración de la columna de DOIS e ISSNs de las revistas  de los artículos de la UdeA
* oadoi.org para la identificación de artículos Open Access
* DOAJ para obetener la información del APC basada en la identificación de los ISSNs de las revistas
* Búsqueda en google scholar (GS) de cada artículo basada en DOI y en título

In [1]:
import wosplus as wp

In [2]:
%%writefile drive.cfg
[FILES]
DOIS.xlsx               = 1bikNT7Gmp4G7dfeMuGsF-az7D8lskK0O
UDEA_WOS_SCI_SCP.xlsx   = 1o9otmklgh-0w18Avv2ZTKOXr3vZbjwvj
oaudea.xlsx             = 1CcwobiEFACIbffNzNdLxpdxQukr8cZ5x
datos1.csv              = 11CyLRZZwVbgw6YAC-igRJ3mrkIwk0aaiXnd-EOofYTI
oa_doi.json             = 1pLaEtyM-aOhjsB6twr05OhyJToIM7h7e
oaudea.json             = 1BmqRoQDgfpOFjXBVEfI999uaN6XX6Fkd

Overwriting drive.cfg


In [3]:
oa=wp.wosplus('drive.cfg')

In [4]:
df=oa.read_drive_json('oaudea.json')

In [5]:
df.shape

(7990, 39)

In [6]:
df.loc[[1]]

,APC,APC_USD,DOI,GS_Journal,GS_PDF,GS_Year,GS_abstract,GS_authors,GS_cites,GS_cites_link,...,journal_issns,journal_name,language,oa_locations,published_date,publisher,title,updated,year,z_authors
1,0NOT,0,10.1016/s0370-2693(99)01040-0,Lorenzana,,-1,Skip to main content \n,A Perez,6,/scholar?cites=7204515635894942882&as_sdt=2005...,...,0370-2693,Physics Letters B,en,[{'url_for_pdf': 'http://arxiv.org/pdf/hep-ph/...,1999-10-01,Elsevier BV,An [SU(3)]4 supersymmetric grand unified model,2018-06-17T11:59:16.138320,1999,"[{'family': 'Pérez-Lorenzana', 'given': 'Abdel..."


Fix GS_cites

## Análisis

Número de artículos Open Access

In [7]:
dfoa=df[df.Open_Access!=''].reset_index(drop=True)
dfc=df[df.Open_Access==''].reset_index(drop=True) # close 
dfoa.shape[0],dfc.shape[0]

(3628, 4362)

Gasto en OA

In [8]:
df.APC_USD.sum()

1001070

In [9]:
dfoa.APC_USD.sum()

1001070

Citas OA

In [10]:
dfoa.GS_cites.sum()

73963

Falta bajar para no OA ?

In [11]:
dfc.GS_cites.sum()

75002

In [12]:
REBUILD=False
GSYES=dfoa
GSNOT=dfc

In [13]:
if REBUILD:
    import pandas as pd

    noa=pd.read_json('gasnoa.json')

    noa.shape

    GSNOT=GSNOT.drop(['GS_Journal', 'GS_Year', 'GS_abstract', 'GS_authors',
           'GS_cites', 'GS_cites_link', 'GS_profiles',  'GS_title',
           'GS_PDF','GS_old_title','GS_quality_author','GS_quality_title','GS_ref'],axis='columns')

    noags=wp.columns_add_prefix(noa,'GS').rename({'GS_DOI':'DOI'},axis='columns')

    GSNOTgs=GSNOT[GSNOT.DOI!=''].merge(noags[noags.DOI!=''],on='DOI',how='left').reset_index(drop=True)

    GSNOTgs=GSNOTgs.drop_duplicates('DOI').reset_index(drop=True)

    GSNOTgs['GS_cites']=GSNOTgs.GS_cites.fillna(0).astype(int)

    kk=GSYES.append( GSNOTgs,sort=True ).reset_index(drop=True)

    kk.fillna('').to_json('oaudea.json')